# Delta method (simple case)

- This notebook was written by Evangelos Paschalidis.
- Acknowledgement of the material is required.
- The code comes with no guarantees and the author cannot be held liable for any errors.
- Contact: evangelos.paschalidis@epfl.ch

## Overview

This notebook covers a simple application of the Delta method process:

* Load necessary packages
* Define the Delta method function
* Calculations and results

## Context: Calculate the t-ratio of the value of time (e.g. beta_tt/beta_tc)

We have estimated the travel time and travel cost coefficients of a mode choice model.

We have the t-ratios of the individual parameters but not the ratio beta_tt/beta_tc that represents the value of time.

We can calculate the t-ratio of the fraction with the Delta method

## Load packages

Before we estimate the model let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [ ]:
import pandas as pd # for data frame processing
import numpy as np # for some statistical procedures
import numdifftools as nd # we use this to calculate the Jacobian matrix

## Load the data

Now it is time to load the data. We can do that using the piece of code below.

We need to load the results of a previous model and the variance-covariance matrix from the same model.

Both are saved automatically when we estimate a model.

In [ ]:
# Parameter estimates from a model
betas = pd.read_csv('MNL_model_results.csv')

# Variance-covariance matrix from a model
se = pd.read_csv('MNL_model_varcovar.csv', header = None)

# Print the data

## Print the parameter estimates results first

In [ ]:
print(betas)

## Print the variance-covariance matrix

In [ ]:
print(se)

# Function to evaluate

We need to define a function to evaluate. In this case, our function is the ratio of parameters.

**The indexes 0 and 1 of the parameters are not known in advance. We must check them once we load the results**

In [ ]:
def f_eval(beta):
  
  f_result = beta[1]/beta[0]
  
  return(f_result)

# Define the Delta method function

In [ ]:
def deltaMethod(f_eval, betas_filt, se_delta):
  
  jacobian_func = nd.Jacobian(f_eval)
  J = jacobian_func(betas_filt)
  
  newse = J @ se_delta @ J.T
  
  return(newse)

# Data manipulation

We do some data manipulation to facilitate our calculations

In [ ]:
# We define the beta names of the parameters that we need.
betas_names = np.array(['beta_tc', 'beta_tt'])

# We keep only the rows of the results that we need based on the names that we specified above.
betas_filt = betas[betas['Parameter'].isin(betas_names)]

# We extract the indexes (positions)
indexes = betas_filt.index.to_numpy()

# We keep only the values of the parameter estimates.
betas_filt = betas_filt['Estimate'].values

# We extract the respective rows/ columns from the variance-covariance matrix.
se_delta = se.iloc[indexes, indexes]

# Implement the Delta method

## Run the Delta method function

In [ ]:
newse = deltaMethod(f_eval, betas_filt, se_delta)

## Get and print the result (t-ratio value)

In [ ]:
tratio = f_eval(betas_filt)/(np.sqrt(newse))
print(tratio)